In [1]:
from pathlib import Path
import csv
from itertools import groupby
import h5py
import numpy as np
import sklearn
from sklearn.cluster import KMeans
from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import random
import matplotlib
from matplotlib import pyplot as plt

# Configuration & Utilities

In [2]:
random.seed(42)
COLORS = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728", "#9467bd", "#8c564b", "#e377c2", "#7f7f7f", "#bcbd22", "#17becf"]
ANNOTATIONS_PATH = "/media/xtrem/data/experiments/nicolingua-0001-language-id/language-id-annotations/metadata.csv"
FEATURE_DIRS = [
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/wav2vec_features-z',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-c',
    '/media/xtrem/data/experiments/nicolingua-0001-language-id/retrained-wav2vec_features-z'
]

In [3]:
annotation_specification = {
    0: {
        'id': 0,
        'label': "maninka",
        'required_tags': set(['ct-speech', 'lng-maninka']),
        'forbidden_tags':  set(['lng-susu', 'lng-pular'])
    },
    1: {
        'id': 1,
        'label': "susu",
        'required_tags': set(['ct-speech', 'lng-susu']),
        'forbidden_tags':  set(['lng-maninka', 'lng-pular'])
    },
    2: {
        'id': 2,
        'label': "pular",
        'required_tags': set(['ct-speech', 'lng-pular']),
        'forbidden_tags':  set(['lng-susu', 'lng-maninka'])
    }
}

In [4]:
def to_user_friendly_feature_name(fv_name):
    name = fv_name \
        .replace("features-", "") \
        .replace("wav2vec_", "") \
        .replace("average", "avg") \
        .replace("timestep", "T") \
        .replace("c.", "Context") \
        .replace("z.", "Latent")
    return name

# Load annotations

In [5]:
def load_annotations(a_file_path, a_specification):
    with open(ANNOTATIONS_PATH) as f:
        reader = csv.DictReader(f)
        for row in reader:
            tag_set = set([t.strip() for t in row['tags'].split(";")])
            for label in annotation_specification.keys():
                spec = annotation_specification[label]
                if spec['required_tags'].issubset(tag_set):
                    if spec['forbidden_tags'].isdisjoint(tag_set):
                        yield row['file'], label
                        break

data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
audio_files, audio_labels = zip(*data)

## Inspect label counts

In [6]:
def inspect_label_counts():
    for label in annotation_specification:
        count = len([l for l in audio_labels if l == label])
        print("{:10} ({}): {}".format(
            annotation_specification[label]['label'],
            label, 
            count
        ))
inspect_label_counts()

maninka    (0): 114
susu       (1): 32
pular      (2): 28


## Balance data

In [7]:
count_per_class = 28
data = list(load_annotations(ANNOTATIONS_PATH, annotation_specification))
balanced_data = []
for label in annotation_specification:
    balanced_data.extend([d for d in data if d[1] == label][:count_per_class])
audio_files, audio_labels = zip(*balanced_data)

In [8]:
inspect_label_counts()

maninka    (0): 28
susu       (1): 28
pular      (2): 28


# Prepare 10 cross validation folds

In [9]:
TRAIN_PERCENT = .6
FOLD_COUNT = 10

n = len(audio_files)
n_train = int(np.ceil(n * .6))
n_test = n - n_train
all_indices = range(n)

cv_folds = {}
train_count_by_index = {i:0 for i in all_indices}
test_count_by_index = {i:0 for i in all_indices}

for fold_index in range(FOLD_COUNT):
    fold_rsampler = np.random.RandomState(seed=fold_index)
    train_index_set = set(fold_rsampler.choice(all_indices, n_train, replace=False))
    test_index_set = set(all_indices).difference(train_index_set)
        
    cv_folds[fold_index] = {
        'train_indices': sorted(list(train_index_set)),
        'test_indices': sorted(list(train_index_set)),
    }


# Load features

In [10]:
def load_features(audio_files, features_input_dir):
    id_list = []
    features_list = []

    for audio_file_name in audio_files:
        feature_file_name = audio_file_name.replace(".wav", ".h5context")
        feature_path = Path(features_input_dir) / feature_file_name
        with h5py.File(feature_path, 'r') as f:
            features_shape = f['info'][1:].astype(int)
            features = np.array(f['features'][:]).reshape(features_shape)
            # features = pool_feature_last_seq(features)
            features_list.append(features)
    return features_list

In [11]:
raw_features = {}
for feature_dir in FEATURE_DIRS:
    feature_name = Path(feature_dir).stem
    raw_features[feature_name] = load_features(audio_files, feature_dir)

## Inspect feature shapes

In [12]:
for feature_name in raw_features.keys():
    print("feature_name: {}. feature shape: {}".format(
        to_user_friendly_feature_name(feature_name),
        raw_features['wav2vec_features-c'][0].shape
    ))

feature_name: c. feature shape: (2998, 512)
feature_name: z. feature shape: (2998, 512)
feature_name: retrained-c. feature shape: (2998, 512)
feature_name: retrained-z. feature shape: (2998, 512)


## Extract feature vectors

In [13]:
def extract_last_timestep_features(raw_features):
    return raw_features[-1, :]

def extract_neuron_average_features(raw_features):
    return np.mean(raw_features, axis=0)

def identity(x):
    return x

feature_extractors = {
    'last_timestep': extract_last_timestep_features,
    'neuron_average': extract_neuron_average_features,
    'raw_features': identity
}

In [14]:
feature_vectors = {}
for feature_name in raw_features.keys():
    for feature_extractor_name in feature_extractors.keys():
        fv_name = f"{feature_name}__{feature_extractor_name}"
        feature_vectors[fv_name] = []
        for f in raw_features[feature_name]:
            feature_vectors[fv_name].append(
                feature_extractors[feature_extractor_name](f)
            )
            
        feature_vectors[fv_name] = np.array(feature_vectors[fv_name])

## Inspect feature vectors

In [15]:
for fvname in feature_vectors.keys():
    print(fvname, feature_vectors[fvname][0].shape)

wav2vec_features-c__last_timestep (512,)
wav2vec_features-c__neuron_average (512,)
wav2vec_features-c__raw_features (2998, 512)
wav2vec_features-z__last_timestep (512,)
wav2vec_features-z__neuron_average (512,)
wav2vec_features-z__raw_features (2998, 512)
retrained-wav2vec_features-c__last_timestep (512,)
retrained-wav2vec_features-c__neuron_average (512,)
retrained-wav2vec_features-c__raw_features (2998, 512)
retrained-wav2vec_features-z__last_timestep (512,)
retrained-wav2vec_features-z__neuron_average (512,)
retrained-wav2vec_features-z__raw_features (2998, 512)


# Classification Models

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from pytorch_model_summary import summary
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim

In [17]:
class LangIdConvNet(nn.Module):
    def __init__(self):
        super(LangIdConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=512, out_channels=1, kernel_size=3)
        self.drop1 = nn.Dropout(p=0.3)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv2 = nn.Conv1d(in_channels=1, out_channels=3, kernel_size=3)
        self.drop2 = nn.Dropout(p=0.3)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv3 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.drop3 = nn.Dropout(p=0.3)
        self.pool3 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        self.conv4 = nn.Conv1d(in_channels=3, out_channels=3, kernel_size=3)
        self.pool4 = nn.MaxPool1d(kernel_size=2, stride=2)
        
    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = self.conv1(x)
        x = F.elu(x)
        x = self.drop1(x)
        x = self.pool1(x)
        
        x = self.conv2(x)
        x = F.elu(x)
        x = self.drop2(x)
        x = self.pool2(x)
        
        x = self.conv3(x)
        x = F.elu(x)
        x = self.drop3(x)
        x = self.pool3(x)
        
        x = self.conv4(x)
        x = F.elu(x)
        x = self.pool4(x)
        
        x = torch.mean(x, dim=2)
        
        return x
        

# Train Classification Models

In [18]:
def get_data_for_fold(fold_id, feature_name, batch_size):
    train_indices = cv_folds[fold_id]['train_indices']
    test_indices = cv_folds[fold_id]['test_indices']

    train_x = np.take(raw_features[feature_name], train_indices, axis=0)
    train_y = np.take(audio_labels, train_indices, axis=0)

    test_x = np.take(raw_features[feature_name], test_indices, axis=0)
    test_y = np.take(audio_labels, test_indices, axis=0)
    
    return train_x, train_y, test_x, test_y

    
def get_loaders_for_fold(fold_id, feature_name, batch_size):
    
    train_x, train_y, test_x, test_y = get_data_for_fold(fold_id, feature_name, batch_size)
    
    train_dataset = TensorDataset(
        torch.tensor(train_x), 
        torch.tensor(train_y)
    )

    train_loader = DataLoader(train_dataset, batch_size=batch_size)

    test_dataset = TensorDataset(
        torch.tensor(test_x), 
        torch.tensor(test_y)
    )

    test_loader = DataLoader(test_dataset, batch_size=batch_size)
    
    return train_loader, test_loader


def get_predictions_for_logits(logits):
    probs = F.softmax(logits, dim=1)
    return torch.argmax(probs, dim=1)

In [19]:
def train_on_fold(fold_id, feature_name, batch_size, epochs):
    torch.manual_seed(0)
    results = {}
    
    model = LangIdConvNet()

    train_loader, test_loader = get_loaders_for_fold(fold_id, feature_name, batch_size)

    print(summary(LangIdConvNet(), torch.zeros((10, 2998, 512)), show_input=False))

    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, epochs+1):
        model.train()
        train_loss = 0
        pred_train_classes = []
        true_train_classes = []

        for batch_idx, (x, y) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(x)
            pred_train_classes.extend(
                get_predictions_for_logits(outputs)
            )
            true_train_classes.extend(y)
            loss = criterion(outputs, y)

            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss = train_loss/len(train_loader)
        train_acc = sklearn.metrics.accuracy_score(true_train_classes, pred_train_classes)


        pred_test_classes = []
        true_test_classes = []
        model.eval()
        test_loss = 0
        for batch_idx, (x, y) in enumerate(test_loader):
            outputs = model(x)

            pred_test_classes.extend(
                get_predictions_for_logits(outputs)
            )

            true_test_classes.extend(y)

            loss = criterion(outputs, y)
            test_loss += loss.item()


        test_loss = test_loss / len(test_loader)
        test_acc = sklearn.metrics.accuracy_score(true_test_classes, pred_test_classes)

        if epoch%10==0:
            print(f"Epoch: {epoch}. Train Loss: {train_loss:0.4}. Test Loss: {test_loss:0.4}. Train Acc: {train_acc:0.4}. Test Acc:{test_acc:0.4}")
            
        results[epoch] = {
            'train_loss': train_loss,
            'test_loss': test_loss,
            'train_acc': train_acc,
            'test_acc': test_acc
        }
        
    return results
    

In [20]:
import csv
from pathlib import Path

def save_results(all_results):
    for result_entry in all_results:
        feature_name = result_entry['feature_name']
        fold_index = result_entry['fold_index']
        Path('cnn_results').mkdir(exist_ok=True)
        fname = f"cnn_results/{feature_name}_{fold_index}.csv"
        with open(fname, 'w') as f:
            writer = csv.writer(f)
            writer.writerow(
                ['epoch', 'train_loss', 'test_loss', 'train_acc', 'test_acc']
            )
            
            for epoch in sorted(result_entry['epochs'].keys()):
                writer.writerow([
                    epoch,
                    result_entry['epochs'][epoch]['train_loss'],
                    result_entry['epochs'][epoch]['test_loss'],
                    result_entry['epochs'][epoch]['train_acc'],
                    result_entry['epochs'][epoch]['test_acc']
                ])

In [21]:
all_results = []

for fold_index in cv_folds:
    for feature_name in raw_features:
        resutls = train_on_fold(fold_index, feature_name, batch_size=10, epochs=500)
        all_results.append({
            'fold_index': fold_index,
            'feature_name': feature_name,
            'epochs': resutls
        })
        save_results(all_results)

-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 1, 2996]           1,537           1,537
         Dropout-2       [10, 1, 2996]               0               0
       MaxPool1d-3       [10, 1, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              12              12
         Dropout-5       [10, 3, 1496]               0               0
       MaxPool1d-6        [10, 3, 748]               0               0
          Conv1d-7        [10, 3, 746]              30              30
         Dropout-8        [10, 3, 746]               0               0
       MaxPool1d-9        [10, 3, 373]               0               0
         Conv1d-10        [10, 3, 371]              30              30
      MaxPool1d-11        [10, 3, 185]               0               0
Total params: 1,609
Trainable params: 1,609
Non-trainable params: 0
--------

Epoch: 200. Train Loss: 0.7034. Test Loss: 0.6973. Train Acc: 0.4902. Test Acc:0.4902
Epoch: 210. Train Loss: 0.6972. Test Loss: 0.6781. Train Acc: 0.5686. Test Acc:0.5882
Epoch: 220. Train Loss: 0.6714. Test Loss: 0.6677. Train Acc: 0.5882. Test Acc:0.5686
Epoch: 230. Train Loss: 0.6565. Test Loss: 0.6506. Train Acc: 0.549. Test Acc:0.5882
Epoch: 240. Train Loss: 0.6379. Test Loss: 0.6349. Train Acc: 0.5686. Test Acc:0.5686
Epoch: 250. Train Loss: 0.6277. Test Loss: 0.6194. Train Acc: 0.5294. Test Acc:0.6078
Epoch: 260. Train Loss: 0.6098. Test Loss: 0.6073. Train Acc: 0.6078. Test Acc:0.5882
Epoch: 270. Train Loss: 0.5958. Test Loss: 0.5966. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 280. Train Loss: 0.5868. Test Loss: 0.5845. Train Acc: 0.6471. Test Acc:0.6275
Epoch: 290. Train Loss: 0.5673. Test Loss: 0.5753. Train Acc: 0.6275. Test Acc:0.6471
Epoch: 300. Train Loss: 0.5639. Test Loss: 0.5678. Train Acc: 0.6471. Test Acc:0.6471
Epoch: 310. Train Loss: 0.5548. Test Loss: 0.5544. Trai

Epoch: 10. Train Loss: 1.123. Test Loss: 1.123. Train Acc: 0.3529. Test Acc:0.3529
Epoch: 20. Train Loss: 1.115. Test Loss: 1.114. Train Acc: 0.3529. Test Acc:0.3529
Epoch: 30. Train Loss: 1.095. Test Loss: 1.097. Train Acc: 0.3529. Test Acc:0.3529
Epoch: 40. Train Loss: 1.065. Test Loss: 1.063. Train Acc: 0.451. Test Acc:0.4902
Epoch: 50. Train Loss: 1.031. Test Loss: 1.023. Train Acc: 0.4902. Test Acc:0.5098
Epoch: 60. Train Loss: 0.9929. Test Loss: 0.9838. Train Acc: 0.4314. Test Acc:0.3922
Epoch: 70. Train Loss: 0.9552. Test Loss: 0.9445. Train Acc: 0.3922. Test Acc:0.4314
Epoch: 80. Train Loss: 0.9211. Test Loss: 0.9055. Train Acc: 0.451. Test Acc:0.4314
Epoch: 90. Train Loss: 0.884. Test Loss: 0.8682. Train Acc: 0.4706. Test Acc:0.5294
Epoch: 100. Train Loss: 0.8446. Test Loss: 0.8312. Train Acc: 0.5294. Test Acc:0.5686
Epoch: 110. Train Loss: 0.8125. Test Loss: 0.7947. Train Acc: 0.549. Test Acc:0.5882
Epoch: 120. Train Loss: 0.7721. Test Loss: 0.7579. Train Acc: 0.5686. Test Ac

Epoch: 330. Train Loss: 0.4295. Test Loss: 0.4847. Train Acc: 0.8627. Test Acc:0.7059
Epoch: 340. Train Loss: 0.418. Test Loss: 0.4687. Train Acc: 0.8824. Test Acc:0.7255
Epoch: 350. Train Loss: 0.4075. Test Loss: 0.4457. Train Acc: 0.902. Test Acc:0.7647
Epoch: 360. Train Loss: 0.3926. Test Loss: 0.4384. Train Acc: 0.902. Test Acc:0.7647
Epoch: 370. Train Loss: 0.3958. Test Loss: 0.3942. Train Acc: 0.8627. Test Acc:0.7647
Epoch: 380. Train Loss: 0.3719. Test Loss: 0.3858. Train Acc: 0.902. Test Acc:0.7647
Epoch: 390. Train Loss: 0.3772. Test Loss: 0.373. Train Acc: 0.902. Test Acc:0.8039
Epoch: 400. Train Loss: 0.3487. Test Loss: 0.3887. Train Acc: 0.9216. Test Acc:0.7647
Epoch: 410. Train Loss: 0.3928. Test Loss: 0.2917. Train Acc: 0.8627. Test Acc:0.902
Epoch: 420. Train Loss: 0.3333. Test Loss: 0.3493. Train Acc: 0.902. Test Acc:0.8235
Epoch: 430. Train Loss: 0.3185. Test Loss: 0.3607. Train Acc: 0.9216. Test Acc:0.7843
Epoch: 440. Train Loss: 0.3064. Test Loss: 0.3494. Train Acc: 

Epoch: 10. Train Loss: 1.13. Test Loss: 1.126. Train Acc: 0.3725. Test Acc:0.3725
Epoch: 20. Train Loss: 1.089. Test Loss: 1.073. Train Acc: 0.3922. Test Acc:0.3922
Epoch: 30. Train Loss: 1.01. Test Loss: 0.9856. Train Acc: 0.4706. Test Acc:0.4314
Epoch: 40. Train Loss: 0.9431. Test Loss: 0.9199. Train Acc: 0.4706. Test Acc:0.4706
Epoch: 50. Train Loss: 0.8826. Test Loss: 0.8592. Train Acc: 0.5098. Test Acc:0.5098
Epoch: 60. Train Loss: 0.8246. Test Loss: 0.7988. Train Acc: 0.549. Test Acc:0.549
Epoch: 70. Train Loss: 0.7679. Test Loss: 0.7543. Train Acc: 0.5686. Test Acc:0.549
Epoch: 80. Train Loss: 0.7288. Test Loss: 0.7213. Train Acc: 0.5686. Test Acc:0.5686
Epoch: 90. Train Loss: 0.6975. Test Loss: 0.6922. Train Acc: 0.5686. Test Acc:0.5686
Epoch: 100. Train Loss: 0.6649. Test Loss: 0.665. Train Acc: 0.5882. Test Acc:0.5882
Epoch: 110. Train Loss: 0.6347. Test Loss: 0.6363. Train Acc: 0.6078. Test Acc:0.5882
Epoch: 120. Train Loss: 0.6045. Test Loss: 0.612. Train Acc: 0.6275. Test 

...

Epoch: 100. Train Loss: 0.9513. Test Loss: 0.9509. Train Acc: 0.4314. Test Acc:0.3529
Epoch: 110. Train Loss: 0.9209. Test Loss: 0.9228. Train Acc: 0.4902. Test Acc:0.3529
Epoch: 120. Train Loss: 0.8895. Test Loss: 0.8932. Train Acc: 0.5098. Test Acc:0.4314
Epoch: 130. Train Loss: 0.8616. Test Loss: 0.8686. Train Acc: 0.5882. Test Acc:0.451
Epoch: 140. Train Loss: 0.8318. Test Loss: 0.8465. Train Acc: 0.5882. Test Acc:0.451
Epoch: 150. Train Loss: 0.8092. Test Loss: 0.8224. Train Acc: 0.6078. Test Acc:0.4902
Epoch: 160. Train Loss: 0.781. Test Loss: 0.805. Train Acc: 0.6275. Test Acc:0.5098
Epoch: 170. Train Loss: 0.7657. Test Loss: 0.7825. Train Acc: 0.6275. Test Acc:0.5294
Epoch: 180. Train Loss: 0.7431. Test Loss: 0.7659. Train Acc: 0.6275. Test Acc:0.549
Epoch: 190. Train Loss: 0.7267. Test Loss: 0.7428. Train Acc: 0.6471. Test Acc:0.549
Epoch: 200. Train Loss: 0.7. Test Loss: 0.7286. Train Acc: 0.6471. Test Acc:0.549
Epoch: 210. Train Loss: 0.6899. Test Loss: 0.7102. Train Acc: 0.

Epoch: 430. Train Loss: 0.06507. Test Loss: 0.1727. Train Acc: 1.0. Test Acc:0.902
Epoch: 440. Train Loss: 0.05503. Test Loss: 0.1875. Train Acc: 1.0. Test Acc:0.902
Epoch: 450. Train Loss: 0.05364. Test Loss: 0.1785. Train Acc: 1.0. Test Acc:0.902
Epoch: 460. Train Loss: 0.05211. Test Loss: 0.1691. Train Acc: 1.0. Test Acc:0.902
Epoch: 470. Train Loss: 0.04718. Test Loss: 0.1906. Train Acc: 1.0. Test Acc:0.902
Epoch: 480. Train Loss: 0.0452. Test Loss: 0.1586. Train Acc: 1.0. Test Acc:0.902
Epoch: 490. Train Loss: 0.04251. Test Loss: 0.1748. Train Acc: 1.0. Test Acc:0.902
Epoch: 500. Train Loss: 0.03859. Test Loss: 0.1431. Train Acc: 1.0. Test Acc:0.902
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 1, 2996]           1,537           1,537
         Dropout-2       [10, 1, 2996]               0               0
       MaxPool1d-3       [10, 1, 1498]              

Epoch: 120. Train Loss: 0.7669. Test Loss: 0.7714. Train Acc: 0.549. Test Acc:0.549
Epoch: 130. Train Loss: 0.731. Test Loss: 0.7484. Train Acc: 0.5882. Test Acc:0.549
Epoch: 140. Train Loss: 0.711. Test Loss: 0.7228. Train Acc: 0.6078. Test Acc:0.549
Epoch: 150. Train Loss: 0.6758. Test Loss: 0.6978. Train Acc: 0.6471. Test Acc:0.549
Epoch: 160. Train Loss: 0.6518. Test Loss: 0.6759. Train Acc: 0.6471. Test Acc:0.5882
Epoch: 170. Train Loss: 0.6418. Test Loss: 0.6526. Train Acc: 0.6471. Test Acc:0.5882
Epoch: 180. Train Loss: 0.6209. Test Loss: 0.6463. Train Acc: 0.6471. Test Acc:0.5882
Epoch: 190. Train Loss: 0.6075. Test Loss: 0.6261. Train Acc: 0.6471. Test Acc:0.6275
Epoch: 200. Train Loss: 0.5912. Test Loss: 0.6115. Train Acc: 0.6471. Test Acc:0.6275
Epoch: 210. Train Loss: 0.5883. Test Loss: 0.5997. Train Acc: 0.6471. Test Acc:0.6275
Epoch: 220. Train Loss: 0.5788. Test Loss: 0.5918. Train Acc: 0.6471. Test Acc:0.6275
Epoch: 230. Train Loss: 0.5732. Test Loss: 0.6197. Train Acc:

Epoch: 450. Train Loss: 0.3836. Test Loss: 0.4755. Train Acc: 0.9412. Test Acc:0.7451
Epoch: 460. Train Loss: 0.3712. Test Loss: 0.4408. Train Acc: 0.9412. Test Acc:0.7647
Epoch: 470. Train Loss: 0.3748. Test Loss: 0.4296. Train Acc: 0.9804. Test Acc:0.7647
Epoch: 480. Train Loss: 0.3537. Test Loss: 0.4136. Train Acc: 0.9804. Test Acc:0.7647
Epoch: 490. Train Loss: 0.3412. Test Loss: 0.4417. Train Acc: 0.9412. Test Acc:0.7451
Epoch: 500. Train Loss: 0.3309. Test Loss: 0.4409. Train Acc: 0.9608. Test Acc:0.7451
-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
          Conv1d-1       [10, 1, 2996]           1,537           1,537
         Dropout-2       [10, 1, 2996]               0               0
       MaxPool1d-3       [10, 1, 1498]               0               0
          Conv1d-4       [10, 3, 1496]              12              12
         Dropout-5       [10, 3, 1496]               0   

Epoch: 140. Train Loss: 0.6472. Test Loss: 0.6671. Train Acc: 0.6078. Test Acc:0.5882
Epoch: 150. Train Loss: 0.6307. Test Loss: 0.6438. Train Acc: 0.6078. Test Acc:0.6078
Epoch: 160. Train Loss: 0.6095. Test Loss: 0.6296. Train Acc: 0.6275. Test Acc:0.6078
Epoch: 170. Train Loss: 0.6002. Test Loss: 0.6072. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 180. Train Loss: 0.5785. Test Loss: 0.5973. Train Acc: 0.6275. Test Acc:0.6078
Epoch: 190. Train Loss: 0.568. Test Loss: 0.5858. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 200. Train Loss: 0.5588. Test Loss: 0.5778. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 210. Train Loss: 0.5583. Test Loss: 0.565. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 220. Train Loss: 0.543. Test Loss: 0.5599. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 230. Train Loss: 0.5333. Test Loss: 0.5497. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 240. Train Loss: 0.5267. Test Loss: 0.5461. Train Acc: 0.6275. Test Acc:0.6275
Epoch: 250. Train Loss: 0.5342. Test Loss: 0.5371. Train 

In [22]:
LangIdConvNet()

LangIdConvNet(
  (conv1): Conv1d(512, 1, kernel_size=(3,), stride=(1,))
  (drop1): Dropout(p=0.3, inplace=False)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(1, 3, kernel_size=(3,), stride=(1,))
  (drop2): Dropout(p=0.3, inplace=False)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv1d(3, 3, kernel_size=(3,), stride=(1,))
  (drop3): Dropout(p=0.3, inplace=False)
  (pool3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv1d(3, 3, kernel_size=(3,), stride=(1,))
  (pool4): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)